### Load dataset

In [53]:
# from google.colab import drive
# drive.mount("/contetnt/drive")

In [54]:
import numpy as np
import pandas as pd
from os import listdir, makedirs
from os.path import isfile, join, exists

List of activities

In [55]:
# mypath = "/content/drive/Mydrive/data"
mypath = "data"
activities = listdir(mypath)
activities

['idle', 'running', 'stairs', 'walking']

Number of activities

In [56]:
for act in activities:
    path = join(mypath, act)
    frames = listdir(path)
    print(act, ":  ", len(frames))

idle :   1039
running :   3408
stairs :   165
walking :   1850


Data visualization

In [57]:
frames = listdir(join(mypath, activities[-1]))  # "walking"
frame = pd.read_csv(join(join(mypath, activities[-1]), frames[0]))
frame

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,2.322376,3.643975,11.827356
1,1.781286,4.668694,6.148310
2,7.963115,-4.113238,1.359907
3,1.996764,-1.398214,1.163582
4,5.185841,-12.143391,4.687847
5,11.219229,-10.625467,-8.087613
6,10.888829,-13.939042,-1.273715
7,9.902418,-9.667787,-0.110133
8,8.097190,-20.934900,1.393425
9,5.655105,-11.573571,-1.106121


### Data processing

In [58]:
import pickle
from scipy.fftpack import fft, rfft
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error

Features

In [59]:
features = [
    "skew_X",
    "skew_Y",
    "skew_Z",
    "kurtosis_X",
    "kurtosis_Y",
    "kurtosis_Z",
    "max_X",
    "max_Y",
    "max_Z",
    "min_X",
    "min_Y",
    "min_Z",
    "mean_X",
    "mean_Y",
    "mean_Z",
    "std_X",
    "std_Y",
    "std_Z",
    "variance_X",
    "variance_Y",
    "variance_Z",
    "median_X",
    "median_Y",
    "median_Z",
    "index_max_X",
    "index_max_Y",
    "index_max_Z",
    "index_min_X",
    "index_min_Y",
    "index_min_Z",
    "correlation_X",
    "correlation_Y",
    "correlation_Z",
    "mae_X",
    "mae_Y",
    "mae_Z",
    "rmse_X",
    "rmse_Y",
    "rmse_Z",
]

len(features)

39

Function to calculate all statistic features

In [60]:
def get_stat_features(frame):
    fr = frame.copy()
    features = []
    features = np.array(features)

    features = np.concatenate((features, fr.skew(axis=0).values), axis=0)
    features = np.concatenate((features, fr.kurt(axis=0).values), axis=0)
    features = np.concatenate((features, fr.max(axis=0).values), axis=0)
    features = np.concatenate((features, fr.min(axis=0).values), axis=0)
    features = np.concatenate((features, fr.mean(axis=0).values), axis=0)
    features = np.concatenate((features, fr.std(axis=0).values), axis=0)
    features = np.concatenate((features, fr.var(axis=0).values), axis=0)
    features = np.concatenate((features, fr.median(axis=0).values), axis=0)
    features = np.concatenate((features, fr.idxmax(axis=0).values), axis=0)
    features = np.concatenate((features, fr.idxmin(axis=0).values), axis=0)

    correlations = fr.corr()
    corr = np.array(
        [
            correlations["accelerometer_X"]["accelerometer_Y"],
            correlations["accelerometer_Y"]["accelerometer_Z"],
            correlations["accelerometer_Z"]["accelerometer_Z"],
        ]
    )
    features = np.concatenate((features, corr), axis=0)

    fr["mean_X"] = fr.mean(axis=0)["accelerometer_X"]
    fr["mean_Y"] = fr.mean(axis=0)["accelerometer_Y"]
    fr["mean_Z"] = fr.mean(axis=0)["accelerometer_Z"]

    mae_X = mean_absolute_error(fr["accelerometer_X"], fr["mean_X"])
    mae_Y = mean_absolute_error(fr["accelerometer_Y"], fr["mean_Y"])
    mae_Z = mean_absolute_error(fr["accelerometer_Z"], fr["mean_Z"])

    rmse_X = np.sqrt(mean_squared_error(fr["accelerometer_X"], fr["mean_X"]))
    rmse_Y = np.sqrt(mean_squared_error(fr["accelerometer_Y"], fr["mean_Y"]))
    rmse_Z = np.sqrt(mean_squared_error(fr["accelerometer_Z"], fr["mean_Z"]))

    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_X, rmse_Y, rmse_Z])
    features = np.concatenate((features, metrics), axis=0)

    return features


len(get_stat_features(frame))

39

In [61]:
stat_features = pd.DataFrame(data=get_stat_features(frame), index=features)
stat_features.columns = ["values"]
stat_features

,values
skew_X,0.197811
skew_Y,0.659964
skew_Z,-1.635672
kurtosis_X,-0.543699
kurtosis_Y,1.078947
kurtosis_Z,7.158918
max_X,12.363658
max_Y,4.668694
max_Z,11.827356
min_X,-2.389413


Function to calculate all Fast Fourier Transform coefficients (convertion of data to frequency domain from time domain)

In [62]:
def get_fft_features(frame):
    fft_X = np.array(preprocessing.normalize([rfft(frame["accelerometer_X"].values)]))
    fft_Y = np.array(preprocessing.normalize([rfft(frame["accelerometer_Y"].values)]))
    fft_Z = np.array(preprocessing.normalize([rfft(frame["accelerometer_Z"].values)]))

    features = np.concatenate((fft_X, fft_Y), axis=0)
    features = np.concatenate((features, fft_Z), axis=0)

    return features.T


len(get_fft_features(frame))

30

In [63]:
fft_features = pd.DataFrame(data=get_fft_features(frame), index=frame.index)
fft_features.columns = [frame.columns]
fft_features

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,0.818215,-0.922151,-0.183724
1,0.079673,0.142657,0.208943
2,-0.259434,0.026101,0.073640
3,-0.266388,0.178724,0.338499
4,-0.013780,-0.071970,-0.301204
5,-0.125891,0.140604,0.160996
6,0.083944,-0.120517,-0.028451
7,-0.016029,-0.017203,0.207865
8,-0.106537,0.033249,0.109112
9,0.003464,0.086365,0.136449


### Data Preparation

Prepare one class with statistic & FFT features

In [64]:
def class_data_stat_fft_prepare(class_name, class_number):
    path = join(mypath, class_name)
    X = []

    for item in listdir(path):
        frame = pd.read_csv(join(path, item))
        features_stat = get_stat_features(frame)
        features_fft = get_fft_features(frame)
        features = np.concatenate((features_stat, features_fft.flatten()), axis=0)
        X.append(features)

    y = [class_number] * len(X)

    X = np.array(X)
    y = np.array(y)

    return X, y

Prepare one class with FFT features

In [65]:
def class_data_fft_prepare(class_name, class_number):
    path = join(mypath, class_name)
    X = []

    for item in listdir(path):
        frame = pd.read_csv(join(path, item))
        features = get_fft_features(frame)
        X.append(features.flatten())

    y = [class_number] * len(X)

    X = np.array(X)
    y = np.array(y)

    return X, y


Prepare one class with statistic features

In [66]:
def class_data_stat_prepare(class_name, class_number):
    path = join(mypath, class_name)
    X = []

    for item in listdir(path):
        frame = pd.read_csv(join(path, item))
        features = get_stat_features(frame)
        X.append(features)

    y = [class_number] * len(X)

    X = np.array(X)
    y = np.array(y)

    return X, y

### Create DataSet

Folder for DataSets

In [67]:
ds_folder = "datasets"
if not exists(ds_folder):
    makedirs(ds_folder)

In [68]:
def create_dataset(class_prepare):
    X_idle, y_idle = class_prepare("idle", 0)
    X_walking, y_walking = class_prepare("walking", 1)
    X_stairs, y_stairs = class_prepare("stairs", 2)
    X_running, y_running = class_prepare("running", 3)

    X = np.concatenate((X_idle, X_walking), axis=0)
    X = np.concatenate((X, X_stairs), axis=0)
    X = np.concatenate((X, X_running), axis=0)

    Y = np.concatenate((y_idle, y_walking), axis=0)
    Y = np.concatenate((Y, y_stairs), axis=0)
    Y = np.concatenate((Y, y_running), axis=0)
    
    return X, Y

DataSet with FFT class features

In [69]:
X_fft, Y_fft = create_dataset(class_data_fft_prepare)

with open(join(ds_folder, "fft_data_X.pickle"), "wb") as fd:
    pickle.dump(X_fft, fd)

with open(join(ds_folder, "fft_data_Y.pickle"), "wb") as fd:
    pickle.dump(Y_fft, fd)

DataSet with statistic class features

In [70]:
X_stat, Y_stat = create_dataset(class_data_stat_prepare)

with open(join(ds_folder, "stat_data_X.pickle"), "wb") as fd:
    pickle.dump(X_stat, fd)

with open(join(ds_folder, "stat_data_Y.pickle"), "wb") as fd:
    pickle.dump(Y_stat, fd)

DataSet with statistic & FFT class features

In [71]:
X_fft_stat, Y_fft_stat = create_dataset(class_data_stat_fft_prepare)

with open(join(ds_folder, "fft_stat_data_X.pickle"), "wb") as fd:
    pickle.dump(X_fft_stat, fd)

with open(join(ds_folder, "fft_stat_data_Y.pickle"), "wb") as fd:
    pickle.dump(Y_fft_stat, fd)

### Load DataSet

flag = "fft" | "stat" | "fft_stat"

In [72]:
def open_data_set(flag):
    name = flag + "_data_"

    with open(join(ds_folder, name + "X.pickle"), "rb") as fd:
        X = pickle.load(fd)

    with open(join(ds_folder, name + "Y.pickle"), "rb") as fd:
        y = pickle.load(fd)

    return X, y

### Results Validation

In [73]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score

In [74]:
def statistic_counter(classifier, X_train, y_train, X_test, y_test):

    def counter(list_item):
        idle = 0
        running = 0
        stairs = 0
        walking = 0

        for i in list_item:
            if i == 0:
                idle += 1
            elif i == 1:
                running += 1
            elif i == 2:
                stairs += 1
            elif i == 3:
                walking += 1

        return idle, running, stairs, walking
    

    scores = cross_val_score(classifier, X_train, y_train, cv=10).mean()

    y_predict = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)

    f1score = f1_score(y_test, y_predict, average="macro")

    print("_________Test vs Prediction_________")
    print(f"Idle:     {counter(y_test)[0]} vs {counter(y_predict)[0]}")
    print(f"Running:  {counter(y_test)[1]} vs {counter(y_predict)[1]}")
    print(f"Stairs:    {counter(y_test)[2]} vs {counter(y_predict)[2]}")
    print(f"Walking: {counter(y_test)[3]} vs {counter(y_predict)[3]}")
    print(f"TOTAL:   {len(y_test)}")
    print("____________________________________")
    print(f"Score:    {scores}")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1score}")
    print("____________________________________")

In [75]:
def accuracy_report(classifier, X_test, y_test):
    y_predict = classifier.predict(X_test)
    target_names = ["Idle", "Running", "Stairs", "Walking"]

    print(classification_report(y_test, y_predict, target_names=target_names))

### SVM Classifier

In [76]:
from sklearn import svm

#### SVM Classifier on Statistic and FFM Features (Time & Frequency Domain)

Data loading

In [77]:
X, y = open_data_set("fft_stat")
X_train_all, X_test_all, y_train_all, y_test_all = model_selection.train_test_split(X, y, test_size=0.3)

Classifier "one vs one"

In [78]:
classifier = svm.SVC(decision_function_shape="ovo", kernel="rbf", gamma=0.005, probability=True)
classifier.fit(X_train_all, y_train_all)

SVC(decision_function_shape='ovo', gamma=0.005, probability=True)

In [79]:
y_test_pred = classifier.decision_function(X_test_all)
y_test_pred

array([[-0.58799416,  0.40441488, -0.98003831,  0.85628191, -0.93498444,
        -0.991338  ],
       [ 1.20062347,  1.07793253,  1.22857049,  0.85624506, -0.54388005,
        -0.93756074],
       [-0.58799416,  0.40441488, -0.93516994,  0.85628191, -0.79171963,
        -0.97144831],
       ...,
       [-0.58799416,  0.40441488, -0.96681009,  0.85628191, -0.89386433,
        -0.98544168],
       [ 1.49408351,  1.18460604,  1.57896149,  0.85620966, -0.54390793,
        -0.93753244],
       [-0.58799448,  0.40441488, -0.96631325,  0.85628201, -0.89127693,
        -0.98495525]])

In [80]:
y_probability = classifier.predict_proba(X_test_all)
y_probability

array([[7.15504008e-04, 7.76977541e-05, 6.87229310e-04, 9.98519569e-01],
       [9.99895660e-01, 1.01036197e-07, 1.03448204e-04, 7.90590551e-07],
       [1.59803664e-03, 2.12905949e-04, 1.17980437e-03, 9.97009253e-01],
       ...,
       [9.06940197e-04, 1.00335642e-04, 8.06012727e-04, 9.98186711e-01],
       [9.99966013e-01, 5.67470801e-08, 3.36188782e-05, 3.11726597e-07],
       [9.15037970e-04, 1.01445638e-04, 8.15871195e-04, 9.98167645e-01]])

In [81]:
y_prediction = np.argmax(y_probability, axis=1)
y_prediction

array([3, 0, 3, ..., 3, 0, 3], dtype=int64)

In [82]:
statistic_counter(classifier, X_train_all, y_train_all, X_test_all, y_test_all)
accuracy_report(classifier, X_test_all, y_test_all)

_________Test vs Prediction_________
Idle:     307 vs 288
Running:  566 vs 512
Stairs:    50 vs 38
Walking: 1016 vs 1101
TOTAL:   1939
____________________________________
Score:    0.9571094375744789
Accuracy: 0.9546157813305828
F1 Score: 0.9339734515354645
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      0.94      0.97       307
     Running       0.99      0.90      0.94       566
      Stairs       1.00      0.76      0.86        50
     Walking       0.92      1.00      0.96      1016

    accuracy                           0.95      1939
   macro avg       0.98      0.90      0.93      1939
weighted avg       0.96      0.95      0.95      1939



Same classifier with scale gamma

In [83]:
classifier = svm.SVC(decision_function_shape="ovo", probability=True)
classifier.fit(X_train_all, y_train_all)

SVC(decision_function_shape='ovo', probability=True)

In [84]:
statistic_counter(classifier, X_train_all, y_train_all, X_test_all, y_test_all)
accuracy_report(classifier, X_test_all, y_test_all)

_________Test vs Prediction_________
Idle:     307 vs 307
Running:  566 vs 616
Stairs:    50 vs 0
Walking: 1016 vs 1016
TOTAL:   1939
____________________________________
Score:    0.9745751040262556
Accuracy: 0.9742135121196493
F1 Score: 0.739424703891709
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       307
     Running       0.92      1.00      0.96       566
      Stairs       0.00      0.00      0.00        50
     Walking       1.00      1.00      1.00      1016

    accuracy                           0.97      1939
   macro avg       0.73      0.75      0.74      1939
weighted avg       0.95      0.97      0.96      1939



c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Same classifier with linear kernel

In [85]:
classifier = svm.SVC(decision_function_shape="ovo", probability=True, kernel="linear")
classifier.fit(X_train_all, y_train_all)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [86]:
statistic_counter(classifier, X_train_all, y_train_all, X_test_all, y_test_all)
accuracy_report(classifier, X_test_all, y_test_all)

_________Test vs Prediction_________
Idle:     307 vs 307
Running:  566 vs 559
Stairs:    50 vs 57
Walking: 1016 vs 1016
TOTAL:   1939
____________________________________
Score:    0.9949168766727228
Accuracy: 0.9943269726663229
F1 Score: 0.9718546209761163
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       307
     Running       1.00      0.98      0.99       566
      Stairs       0.84      0.96      0.90        50
     Walking       1.00      1.00      1.00      1016

    accuracy                           0.99      1939
   macro avg       0.96      0.99      0.97      1939
weighted avg       0.99      0.99      0.99      1939



Same classifier with "one vs rest" decision function

In [87]:
classifier = svm.SVC(probability=True, kernel="linear", gamma=0.005)
classifier.fit(X_train_all, y_train_all)

SVC(gamma=0.005, kernel='linear', probability=True)

In [88]:
statistic_counter(classifier, X_train_all, y_train_all, X_test_all, y_test_all)
accuracy_report(classifier, X_test_all, y_test_all)

_________Test vs Prediction_________
Idle:     307 vs 307
Running:  566 vs 559
Stairs:    50 vs 57
Walking: 1016 vs 1016
TOTAL:   1939
____________________________________
Score:    0.9949168766727228
Accuracy: 0.9943269726663229
F1 Score: 0.9718546209761163
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       307
     Running       1.00      0.98      0.99       566
      Stairs       0.84      0.96      0.90        50
     Walking       1.00      1.00      1.00      1016

    accuracy                           0.99      1939
   macro avg       0.96      0.99      0.97      1939
weighted avg       0.99      0.99      0.99      1939



#### SVM Classifier on Statistic Features (Time Domain)

Data loading

In [89]:
X, y = open_data_set("stat")
X_train_stat, X_test_stat, y_train_stat, y_test_stat = model_selection.train_test_split(X, y, test_size=0.3)

Classifier "one vs rest", linear

In [90]:
classifier = svm.SVC(probability=True, kernel="linear")
classifier.fit(X_train_stat, y_train_stat)

SVC(kernel='linear', probability=True)

In [91]:
statistic_counter(classifier, X_train_stat, y_train_stat, X_test_stat, y_test_stat)
accuracy_report(classifier, X_test_stat, y_test_stat)

_________Test vs Prediction_________
Idle:     318 vs 318
Running:  540 vs 536
Stairs:    46 vs 50
Walking: 1035 vs 1035
TOTAL:   1939
____________________________________
Score:    0.994914923128016
Accuracy: 0.9948427024239299
F1 Score: 0.9716349132589839
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       318
     Running       0.99      0.99      0.99       540
      Stairs       0.86      0.93      0.90        46
     Walking       1.00      1.00      1.00      1035

    accuracy                           0.99      1939
   macro avg       0.96      0.98      0.97      1939
weighted avg       1.00      0.99      0.99      1939



#### SVM Classifier on FFT Features (Frequency Domain)

Data loading

In [92]:
X, y = open_data_set("fft")
X_train_fft, X_test_fft, y_train_fft, y_test_fft = model_selection.train_test_split(X, y, test_size=0.3)

Classifier "one vs rest", linear

In [93]:
classifier = svm.SVC(probability=True, kernel="linear")
classifier.fit(X_train_fft, y_train_fft)

SVC(kernel='linear', probability=True)

In [94]:
statistic_counter(classifier, X_train_fft, y_train_fft, X_test_fft, y_test_fft)
accuracy_report(classifier, X_test_fft, y_test_fft)

_________Test vs Prediction_________
Idle:     307 vs 312
Running:  553 vs 561
Stairs:    47 vs 39
Walking: 1032 vs 1027
TOTAL:   1939
____________________________________
Score:    0.9783342124284514
Accuracy: 0.9767921609076844
F1 Score: 0.8721177926229216
____________________________________
              precision    recall  f1-score   support

        Idle       0.98      1.00      0.99       307
     Running       0.96      0.97      0.96       553
      Stairs       0.59      0.49      0.53        47
     Walking       1.00      1.00      1.00      1032

    accuracy                           0.98      1939
   macro avg       0.88      0.86      0.87      1939
weighted avg       0.98      0.98      0.98      1939



### Random Forest Classifier

In [95]:
from sklearn.ensemble import RandomForestClassifier

#### Random Forest on FFT & Statistic Features

In [96]:
classifier = RandomForestClassifier(n_estimators=15, max_depth=None)
classifier.fit(X_train_all, y_train_all)

RandomForestClassifier(n_estimators=15)

In [97]:
statistic_counter(classifier, X_train_all, y_train_all, X_test_all, y_test_all)
accuracy_report(classifier, X_test_all, y_test_all)

_________Test vs Prediction_________
Idle:     307 vs 307
Running:  566 vs 573
Stairs:    50 vs 43
Walking: 1016 vs 1016
TOTAL:   1939
____________________________________
Score:    0.993812635527164
Accuracy: 0.9963898916967509
F1 Score: 0.9796463602292145
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       307
     Running       0.99      1.00      0.99       566
      Stairs       1.00      0.86      0.92        50
     Walking       1.00      1.00      1.00      1016

    accuracy                           1.00      1939
   macro avg       1.00      0.96      0.98      1939
weighted avg       1.00      1.00      1.00      1939



#### Random Forest on FFT Features

In [98]:
classifier = RandomForestClassifier(n_estimators=15, max_depth=None)
classifier.fit(X_train_fft, y_train_fft)

RandomForestClassifier(n_estimators=15)

In [99]:
statistic_counter(classifier, X_train_fft, y_train_fft, X_test_fft, y_test_fft)
accuracy_report(classifier, X_test_fft, y_test_fft)

_________Test vs Prediction_________
Idle:     307 vs 307
Running:  553 vs 590
Stairs:    47 vs 10
Walking: 1032 vs 1032
TOTAL:   1939
____________________________________
Score:    0.9756793451718142
Accuracy: 0.9798865394533265
F1 Score: 0.8204171846940185
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       307
     Running       0.94      1.00      0.97       553
      Stairs       0.90      0.19      0.32        47
     Walking       1.00      1.00      1.00      1032

    accuracy                           0.98      1939
   macro avg       0.96      0.80      0.82      1939
weighted avg       0.98      0.98      0.97      1939



#### Random Forest on Statistic Features

In [100]:
classifier = RandomForestClassifier(n_estimators=15, max_depth=None)
classifier.fit(X_train_stat, y_train_stat)

RandomForestClassifier(n_estimators=15)

In [101]:
statistic_counter(classifier, X_train_stat, y_train_stat, X_test_stat, y_test_stat)
accuracy_report(classifier, X_test_stat, y_test_stat)

_________Test vs Prediction_________
Idle:     318 vs 318
Running:  540 vs 543
Stairs:    46 vs 43
Walking: 1035 vs 1035
TOTAL:   1939
____________________________________
Score:    0.9982310652679285
Accuracy: 0.998452810727179
F1 Score: 0.9908805129322418
____________________________________
              precision    recall  f1-score   support

        Idle       1.00      1.00      1.00       318
     Running       0.99      1.00      1.00       540
      Stairs       1.00      0.93      0.97        46
     Walking       1.00      1.00      1.00      1035

    accuracy                           1.00      1939
   macro avg       1.00      0.98      0.99      1939
weighted avg       1.00      1.00      1.00      1939



Feature Analysis

In [102]:
feature_importance = classifier.feature_importances_

print("Feature importances scaled (x1000)")

for item, feat in zip(feature_importance, features):
    print(feat, "  -->  ", item*1000)

Feature importances scaled (x1000)
skew_X   -->   0.6376617574489379
skew_Y   -->   0.6772815967443817
skew_Z   -->   0.833827416240516
kurtosis_X   -->   0.2319360709387191
kurtosis_Y   -->   0.46002430744964107
kurtosis_Z   -->   0.6602301573913449
max_X   -->   2.4352211998138324
max_Y   -->   135.21178813182692
max_Z   -->   2.0839304940058128
min_X   -->   14.140298236392434
min_Y   -->   66.3423652966186
min_Z   -->   56.268286786481035
mean_X   -->   3.496987455732717
mean_Y   -->   127.01333227451936
mean_Z   -->   37.972141847042494
std_X   -->   3.3764114570991945
std_Y   -->   39.19315933757605
std_Z   -->   23.835900535512017
variance_X   -->   22.54018995570122
variance_Y   -->   78.40434574930755
variance_Z   -->   26.385626429958165
median_X   -->   9.489441632233596
median_Y   -->   3.221524388690863
median_Z   -->   3.7817749639507805
index_max_X   -->   0.27234522838687875
index_max_Y   -->   0.11356294804818388
index_max_Z   -->   0.21020369881764794
index_min_X   --

#### Conclusion: the best algorithm is the Random Forest on Statistical Features (Time Domain), it is the only one that has Score, Accuracy and F1 higher than 0.99.